In [ ]:
from models import LSTMModel , LSTMModel_seeOnce
from dataset import ImageCaptionDataset
import pickle
from torch.utils.data import DataLoader

import gensim.downloader as api

from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

import torch
import torch.nn as nn

In [ ]:
batch_size = 1

In [ ]:
with open('C:/Users/Júlia Garcia Torné/Desktop/Artificial inteligence/Segon curs/Part 2/Neural Networks and Deep Learning/Project/dlnn-project_ia-group_10/dataset/train_dataset.pkl', 'rb') as inp:
    train_dataset = pickle.load(inp)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)


#load word2vec pretrained embedding layer
word2vec_emb = api.load('word2vec-google-news-300')
word2vec_emb = torch.FloatTensor(word2vec_emb.vectors)

#import vocabulary to word2vec indexes that we know
with open('vocabidx2word2vecidx.pkl', 'rb') as inp:
    vocabidx2word2vecidx = pickle.load(inp)

with open('vocabulary.pkl', 'rb') as inp:
    vocabulary = pickle.load(inp)

word2vec_emb = word2vec_emb[vocabidx2word2vecidx]

word2vec_emb = nn.Embedding.from_pretrained(word2vec_emb)
word2vec_emb.requires_grad_ = False #freeze word2vec embeddding layer

#load universal sentence encoder to define our own loss function
sntc_enc = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

#define cross entropy loss function
cross_entrop = nn.CrossEntropyLoss()

In [ ]:
emb_layer = nn.Embedding(num_embeddings=word2vec_emb.weight.shape[0], embedding_dim=100)

model = LSTMModel(input_dim=512,embedding_layer=emb_layer,hidden_dim=100,n_layers=1)
X1,X2,caption = next(iter(train_dataloader))
out,_,_ = model(X1,X2)

In [ ]:
from torchviz import make_dot

dot = make_dot(out)
dot.render('computation_graph', format='png')

In [ ]:
input_names = ['Image features', "Captions"]
output_names = ['Caption',"Hidden states","Cell states"]
torch.onnx.export(model, (X1,X2), 'lstm.onnx', input_names=input_names, output_names=output_names)